## Fourier series and Data encoding for QML

In this notebook, I perform a literature review and replicate the neccessary codes for the graphs the authors presented in their article [1].

### 1. Introductory ideas
Besides from practical approach of QML, the authors of [this article](https://arxiv.org/pdf/2008.08605.pdf) has shown the data encoding can be encapsulated in [Fourier series](https://mathworld.wolfram.com/FourierSeries.html) [2]. The underlying assumption for such demonstration is the encoder is an unitary gate in the form of $e^{-ixH}$. H is some arbitary Hamiltonian of the system and x is the data to encode. This sums down to evolution of the state by 'x' amount, moreover, if we take H to be a Pauli gate then the evolution turns into rotation of the state. 

The authors then goes on to demonstrate the link between different terms in Fourier series and the elements of the Quantum model. A quick glimpse on what are the links are below.

The series is written as below.

$f_{\theta}(x) = \Sigma_{\omega\in\Omega} c_\omega(\theta)e^{i\omega x}$

The Time evolution of quantum state in Schrodinger's picture becomes pretty handy here. Except for the fact that the evolution is an angle and not time. So, it's all rotation operations [3] [4]. The Hamiltonian operator is replaced with it's eigenvalues and multiplied with the respective eigenvector to evolve the eigenstate. The linear superposition of all such operations are deemed as the evolution of the entire quantum state. We can observe a similar fashion in the fourier series. The frequency spectrum of the Pauli gates is $\Omega \subset \R^N$, in natural setting this always turns out to be set of spectrum is set of Integers i.e. $\Omega \subset \Z^N$. The eigenvalues of the Pauli matrices are $1$ and $-1$, a repeating application of the gate say n-times yields n-rotations of $\pi/2$. So, the Fourier series can be reformulated as below.

$f_{\theta}(x) = \Sigma_{n\in\Omega} c_n(\theta)e^{in x}$

The term $f_{\theta}$ refers to a real value and is actually the bra-ket notation of the quantum model as following. The figure "Figure 1" in coming para helps absorb this notion of the quantum model.

$f_{\theta}(x) = <0|U^\dag (x,\theta) M U(x,\theta)|0>$

Now back to the main concerns of links, the frequency spectrum is solely determined by the data-encoding Hamiltonian (Pauli gates in this case). The rest of the circuit (Quantum model), including the trainable parameters, determines the co-efficients. This means a target state has a rigid spectrum and the rest of the parameters, like the co-efficients, inputs and measurement, can be arbitrarily initiated.

#### 1.1 Objectives
The authors succinctly write their ultimate goal in the article, it's quoted below.

> we study the universality of quantum models. We show that for suﬃciently ﬂexible trainable circuit blocks there exists a quantum model which can realise any possible set of Fourier coeﬃcients. If, asymptotically, the accessible frequency spectrum is rich enough, then such models are universal function approximators. This follows from the fact that Fourier series with arbitrary coeﬃcients can approximate any square integrable function on a given interval.

Before they demonstrate the universality, their approach is to slowly level up the game. This notebook too, will follow in their footsteps. In an one-qubit system, the bareminimum process is to encode the data only once into the angle of a single qubit rotation. This leads to the function class such that the quantum models can only learn up to a simple sine function (or equivalently, a Fourier series with a single frequency). The encoding occurs either sequential or parallel application of the encoding gate(s).

To ease the thought process a schematic from the paper is placed below.

<figure>
<img src="assets/images/qc-model-general.png" alt="Quantum Data encoding model" width="500"/>
<figcaption><b>Figure 1 - Quantum Data encoding model for a multi-qubit system</b></figcaption>
</figure>

The above schematic is general. For the one qubit system it can be reduced into a smaller form like following.

<figure>
<img src="assets/images/qc-model.png" alt="Quantum Data encoding model" width="400"/>
<figcaption><b>Figure 1 - Quantum Data encoding model for a single qubit system</b></figcaption>
</figure>

The expression for the quantum model depicted in "Figure 1" and expressed by the term $f_{\theta}(x)$ in above paragraph boils down as Fourier series with some details like below.

$f_{\theta}(x) = \Sigma_{\omega\in\Omega} c_\omega(\theta)e^{i\omega x}$

here, $\omega = \Lambda_k - \Lambda_j$, this is the frequency determined by the Pauli gates embedded in the circuit. The spectrum is written as $\Omega = \{\Lambda_k - \Lambda_j, s.t., k,j \in [d]^L\}$ 

In essence, the spectrum widens up only upto $2L$ on each positive and negative real number line. Moreover, the section II, "THE EXPRESSIVITY OF QUANTUM MODELS" subsection B "Repeated Pauli encodings linearly extend the frequency spectrum" of the paper shows r repetition of the encoding gate produces the spectrum $\Omega = \{-r, -(r-1), ..., 0, ..., r-1, r\}$. The ensuing inference is quoted below.

>Hence, a univariate quantum model with r parallel Pauli-rotation encodings can be expressed as a truncated Fourier series of degree r.

The authors have provided proof for expressing the quantum model in terms of the Fourier series, however, the proof is loaded with subscript and superscript notations. It starts with eigen decomposition [5] and separation of data dependent terms (exponential forms of the Paulis). Yet, I remain undeterred to attempt it in simpler way with different section of the same paper. In my opinion, the aforementioned Subsection B from Section II is a naive way to traverse from the Quantum model to the Fourier expression.

#### 1.2 Quantum model to Fourier series


### 2. Data encoding

#### 2.1. Single data encoding for a single qubit system

The Layer L is 1 for this motivating exercise. $L = 1$ can be visualised from "Figure 1" above.

## References

1. Schuld, Maria, Ryan Sweke, and Johannes Jakob Meyer. "Effect of data encoding on the expressive power of variational quantum-machine-learning models." Physical Review A 103.3 (2021): 032430.

2.  Weisstein, Eric W. "Fourier Series." From MathWorld--A Wolfram Web Resource. https://mathworld.wolfram.com/FourierSeries.html 

3. Yepez, J. (2013). Lecture notes: Qubit representations and rotations. Phys 711 Topics in Particles &amp; Fields. Retrieved February 15, 2022, from https://www.phys.hawaii.edu/~yepez/Spring2013/lectures/Lecture1_Qubits_Notes.pdf 

4. Physics Stack Exchange. 2019. Exponential of the Pauli matrices. [online] Available at: <https://physics.stackexchange.com/questions/457882/exponential-of-the-pauli-matrices> [Accessed 15 February 2022].

5. Weisstein, Eric W. "Eigen Decomposition." From MathWorld--A Wolfram Web Resource. https://mathworld.wolfram.com/EigenDecomposition.html 